In [1]:
param_date = '20240706'

In [2]:
import os

list_file = sorted(list(os.walk(f'./{param_date}/11.part_2'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

['part_1.Part_1.xlsx',
 'part_10.Part_10.xlsx',
 'part_11.Part_11.xlsx',
 'part_12.Part_12.xlsx',
 'part_13.Part_13.xlsx',
 'part_14.Part_14.xlsx',
 'part_15.Part_15.xlsx',
 'part_16.Part_16.xlsx',
 'part_17.Part_17.xlsx',
 'part_18.Part_18.xlsx',
 'part_19.Part_19.xlsx',
 'part_2.Part_2.xlsx',
 'part_20.Part_20.xlsx',
 'part_21.Part_21.xlsx',
 'part_22.Part_22.xlsx',
 'part_23.Part_23.xlsx',
 'part_24.Part_24.xlsx',
 'part_25.Part_25.xlsx',
 'part_26.Part_26.xlsx',
 'part_27.Part_27.xlsx',
 'part_28.Part_28.xlsx',
 'part_29.Part_29.xlsx',
 'part_3.Part_3.xlsx',
 'part_30.Part_30.xlsx',
 'part_31.Part_31.xlsx',
 'part_32.Part_32.xlsx',
 'part_33.Part_33.xlsx',
 'part_4.Part_4.xlsx',
 'part_5.Part_5.xlsx',
 'part_6.Part_6.xlsx',
 'part_7.Part_7.xlsx',
 'part_8.Part_8.xlsx',
 'part_9.Part_9.xlsx']

In [3]:
file = list_file[0]

file

'part_1.Part_1.xlsx'

In [4]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [5]:
input_ = pd.read_excel(f'./{param_date}/11.part_2/{file}',
                       header=0,
                       dtype=str).fillna('')

input_ = input_[input_['Vehicle Url'] != ''].reset_index(drop=True)

print(f'总数量：{len(input_)}')
input_['Vehicle Url'].tolist()

总数量：2470


['https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=10000&PartType=Suspension%20-%20Control%20Arm%20and%20Ball%20Joint%20Assembly',
 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=100000&PartType=Door%20Lock%20Actuator%20Motor',
 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=100001&PartType=Door%20Lock%20Actuator%20Motor',
 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=100002&PartType=Door%20Lock%20Actuator%20Motor',
 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=100005&PartType=Cable%20-%20Parking%20Brake',
 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=100006&PartType=Cable%20-%20Parking%20Brake',
 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=100007&PartType=Wheel%20Bearing%20Retaining%20Ring',
 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=100008&PartType=Door%20Lock%20Actuator%20Motor',
 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=10001&PartType=Suspen

In [6]:
a = 0

a

0

In [7]:
crawler_status = 'error'

crawler_status

'error'

In [9]:
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

import json

df_temp, i, next_ = pd.DataFrame(), 0, ['Lennon']
while next_:
    try:
        print(i)
        resp = requests.get(f'''{input_.loc[a, 'Vehicle Url']}&start={i*50}&num=50''',
                            headers=get_header(),
                            proxies=get_proxy(),
                            timeout=(10, 10)).text

        # = = = = = = = = = = = = = = =

        soup = BeautifulSoup(resp.strip(), 'lxml')
        html = etree.HTML(str(soup))

        # = = = = = = = = = = = = = = =

        if len(html.xpath('//div[@class="detail-app-pager"]')) == 0:
            continue

        # = = = = = = = = = = = = = = =

        next_ = html.xpath('//a[@id="pagingBottom_nextButton"]')
        if next_ and next_[0].xpath('./@href')[0].strip() == 'Search/#':
            break

        # = = = = = = = = = = = = = = =

        list_th = [text.strip() for text in html.xpath('//th[@scope="col"]/text()')]

        # = = = = = = = = = = = = = = =

        list_tr = html.xpath('//tr[@class="detail-app-row"]')

        # = = = = = = = = = = = = = = =

        df_page = pd.DataFrame({'Part Number': input_.loc[a, 'Part Number'],
                                'Vehicle Url': input_.loc[a, 'Vehicle Url'],
                                'Vehicle Page': i+1,
                                'Row': [j+1 for j in range(len(list_tr))]})

        for j, tr in zip(range(len(list_tr)), list_tr):
            for k, th in zip(range(len(list_th)), list_th):
                df_page.loc[j, th] = tr.xpath(f'./td[{k+1}]/text()')[0].strip()

        df_temp = pd.concat([df_temp, df_page], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        i += 1

    except:
        continue

0
0
0
0
0
0
0
0
0


In [10]:
output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,Part Number,Vehicle Url,Vehicle Page,Row,Year,Make,Model,Configuration,Positions,Application Notes
0,520-389,https://www.dormanproducts.com/itemdetailapp.a...,1,1,2010,Ford,Mustang,,Front Left Lower,To 8/03/09
1,520-389,https://www.dormanproducts.com/itemdetailapp.a...,1,2,2009,Ford,Mustang,,Front Left Lower,
2,520-389,https://www.dormanproducts.com/itemdetailapp.a...,1,3,2008,Ford,Mustang,,Front Left Lower,
3,520-389,https://www.dormanproducts.com/itemdetailapp.a...,1,4,2007,Ford,Mustang,,Front Left Lower,
4,520-389,https://www.dormanproducts.com/itemdetailapp.a...,1,5,2006,Ford,Mustang,,Front Left Lower,
5,520-389,https://www.dormanproducts.com/itemdetailapp.a...,1,6,2005,Ford,Mustang,,Front Left Lower,


In [11]:
crawler_status = 'ok'

crawler_status

'ok'

In [12]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([{'Part Number': input_.loc[a, 'Part Number'],
                             'Vehicle Url': input_.loc[a, 'Vehicle Url']}])

    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

output_error

""


In [13]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['Part Number', 'Vehicle Url', 'Vehicle Page', 'Row'],
                                                ascending=[True, True, True, True],
                                                ignore_index=True)
    output_correct.to_excel('./test_vehicle.xlsx', index=False)

output_correct

,Part Number,Vehicle Url,Vehicle Page,Row,Year,Make,Model,Configuration,Positions,Application Notes
0,520-389,https://www.dormanproducts.com/itemdetailapp.a...,1,1,2010,Ford,Mustang,,Front Left Lower,To 8/03/09
1,520-389,https://www.dormanproducts.com/itemdetailapp.a...,1,2,2009,Ford,Mustang,,Front Left Lower,
2,520-389,https://www.dormanproducts.com/itemdetailapp.a...,1,3,2008,Ford,Mustang,,Front Left Lower,
3,520-389,https://www.dormanproducts.com/itemdetailapp.a...,1,4,2007,Ford,Mustang,,Front Left Lower,
4,520-389,https://www.dormanproducts.com/itemdetailapp.a...,1,5,2006,Ford,Mustang,,Front Left Lower,
5,520-389,https://www.dormanproducts.com/itemdetailapp.a...,1,6,2005,Ford,Mustang,,Front Left Lower,


In [14]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['Part Number', 'Vehicle Url'],
                                            ascending=[True, True],
                                            ignore_index=True)
    output_error.to_excel('./test_vehicle_error.xlsx', index=False)

output_error

""
